Source: https://github.com/BlackPianoCat/EasyOpenMM/tree/main

# Imports

In [1]:
import copy
import numpy as np
import time
from utils import *
import openmm as mm
import openmm.unit as u
from tqdm import tqdm
from sys import stdout
from mdtraj.reporters import HDF5Reporter
from openmm.app import PDBFile, PDBxFile, ForceField, Simulation, PDBReporter, PDBxReporter, DCDReporter, StateDataReporter, CharmmPsfFile,  DCDFile
import random
import pyvista as pv
import mdtraj as md
from hilbert import decode, encode
from initial_structures_defs import *
from model import Model

# Initial structure generation functions

# Simulation

In [13]:
# 0. Generate some initial structure
N_beads=100
points = helisa(N_beads)
write_mmcif(points,'init_struct.cif')
generate_psf(N_beads,'LE_init_struct.psf')

# 1. Define System
pdb = PDBxFile('init_struct.cif')
forcefield = ForceField('forcefields/classic_sm_ff.xml')
system = forcefield.createSystem(pdb.topology, nonbondedCutoff=1*u.nanometer)
integrator = mm.LangevinIntegrator(310, 0.05, 100 * mm.unit.femtosecond)

# 2. Define the forcefield
# 2.1. Harmonic bond borce between succesive beads
bond_force = mm.HarmonicBondForce()
system.addForce(bond_force)
for i in range(system.getNumParticles() - 1):
    bond_force.addBond(i, i + 1, 0.1, 300000.0)
bond_force.addBond(10, 70, 0.001, 0.001) # connect bead 10 with bead 70

#2.2. Harmonic angle force between successive beads so as to make chromatin rigid
angle_force = mm.HarmonicAngleForce()
system.addForce(angle_force)
for i in range(system.getNumParticles() - 2):
    angle_force.addAngle(i, i + 1, i + 2, np.pi, 0.001)
    
# 3. Minimize energy
simulation = Simulation(pdb.topology, system, integrator)
simulation.reporters.append(StateDataReporter(stdout, 10, step=True, totalEnergy=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(DCDReporter('stochastic_LE.dcd', 10))
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy(tolerance=0.001)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('minimized.cif', 'w')) # save minimized file

# 4. Run md simulation
simulation.context.setVelocitiesToTemperature(310, 0)
simulation.step(10000)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.cif', 'w')) # save minimized file
PDBFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.pdb', 'w')) # save minimized file
df = DCDFile(open("after_sim.dcd", "wb"),pdb.topology, dt = 100 * mm.unit.femtosecond)
df.writeModel(state.getPositions(), pdb.topology.getUnitCellDimensions(), pdb.topology.getPeriodicBoxVectors())

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,107.27322387695312,316.13366567716,169.15917313518588
20,82.96114349365234,320.29261162132025,192.21828011759123
30,89.17096710205078,337.1468686237931,200.8393647802227
40,94.56158447265625,336.2572063356638,195.75287303016904
50,91.58721923828125,357.9594160877168,215.7387975285268
60,82.23358154296875,343.2710475176573,211.4181197788637
70,81.62876892089844,351.0761366188526,218.22942406125458
80,96.01492309570312,356.9387115314603,211.32605065211126
90,87.42857360839844,379.08236931450665,236.2147398432575
100,103.54850006103516,391.5979905053973,233.29555949226028
110,87.23733520507812,406.9284926354885,258.9226154241774
120,89.27845764160156,412.5447427332401,261.81816440326367
130,103.89022827148438,423.42676820605993,258.7973884184208
140,104.93695831298828,419.017266407609,254.37830523289568
150,127.93624877929688,434.52535669505596,248.31107097290794
160,106.31491088867188,454.4865112155676,281

# Visualization

In [14]:
traj = md.load("after_sim.cif")

positions = traj.xyz

mesh = pv.PolyData(positions[0])

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions[0])

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f9595e7a00_1&reconnect=auto" class="pyvis…

### Testing initial structure generation function

In [15]:
positions = cube_outer(1000)

mesh = pv.PolyData(positions)

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions)

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f95806f820_2&reconnect=auto" class="pyvis…

In [25]:
positions = hilbert_curve3d(4000)

mesh = pv.PolyData(positions)

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions)

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

c:\Users\Micha\anaconda3\envs\wb\lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f947e019d0_8&reconnect=auto" class="pyvis…

# Testing multiple initial structures

In [2]:
initial_structure_gen_list = [line,random_walk,random_walk2,hilbert_curve2d, hilbert_curve3d, helisa, sphere, cube, cube_outer]

In [4]:
import os
def make_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

for f in initial_structure_gen_list:
    make_directory_if_not_exists(f'initial_structures_tests/{f.__name__}')

In [3]:
for f in initial_structure_gen_list:
    #run_simulation_on_initial_structure(f,100)
    model = Model(200, 30, struct=f)
    model.run_simulation()

/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,868480640.0,2610706871.2386017,701982083.2438178
20,2143629952.0,2512740844.0835648,148723069.55715856
30,1704305152.0,2470520863.6015015,308725521.2899523
40,602766848.0,2343192472.296753,701256578.2608951
50,909325888.0,2179364760.506897,511727189.92366153
60,1478682368.0,2092287531.652893,247235303.5139389
70,1018951616.0,2017725137.4835205,402428286.71068585
80,575784832.0,1898585588.5274963,532986138.1569904
90,926696896.0,1782312835.696167,344746881.3378662
100,1194969472.0,1720850927.1108398,211889450.85251698
110,718912960.0,1650060500.4035645,375180259.8880355
120,560235776.0,1550476802.9239502,398990352.99186254
130,819149312.0,1465668728.8291016,260497195.45355406
140,893602304.0,1412637390.0097656,209130894.95543844
150,528794528.0,1346382884.8945312,329424714.03423136
160,522251712.0,1265145726.3151855,299328685.590727
170,676799040.0,1201587042.241455,211448874.12980634
180,685423296.0,1154

/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,2941.544921875,3548.81727739051,244.68367286497133
20,2982.9404296875,3562.229839220643,233.40871536301944
30,3094.9501953125,3577.9931744374335,194.62886662046935
40,3230.0908203125,3622.3350931499153,158.04402829540206
50,3211.913818359375,3659.53415863961,180.35628975052006
60,3122.1318359375,3709.0219828635454,236.47122318983912
70,3102.250244140625,3691.3111972045153,237.34588974441138
80,3095.971923828125,3733.1245937794447,256.7231227363684
90,3102.96337890625,3739.7139319628477,256.56110080695237
100,3105.533203125,3738.1818586215377,254.90835414131607
110,3146.771484375,3765.7098975367844,249.38418953894669
120,3163.69287109375,3779.2244498636574,248.01149943659627
130,3170.34619140625,3792.6635466106236,250.74564118723842
140,3183.4248046875,3786.322841927409,242.92116177380575
150,3140.482421875,3782.764571480453,258.789905309723
160,3136.220703125,3815.8093963116407,273.8215497151285
170,3169

/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


...

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,2976.927978515625,3532.8665451966226,224.0001362606404
20,3019.01318359375,3561.7463333420455,218.6793771523752
30,3131.66748046875,3581.4573104102165,181.23042586709354
40,3220.56103515625,3618.056792065501,160.15996918916778
50,3166.492431640625,3642.4362760186195,191.768465766935
60,3129.6650390625,3660.259308135137,213.78834945909514
70,3146.599365234375,3669.834787392989,210.82330473966982
80,3084.67041015625,3712.420378893614,252.93456320951552
90,3121.849853515625,3717.595847844146,240.03944303236767
100,3151.017578125,3754.0375327914953,242.970285043542
110,3165.490234375,3773.8613732606173,245.1263973661593
120,3183.89990234375,3789.063617967069,243.83405448717969
130,3193.32568359375,3817.0949102472514,251.33063280668867
140,3200.9404296875,3841.083347674459,257.92796083823856
150,3181.0576171875,3855.7104139067233,271.83276615605524
160,3172.18310546875,3857.7233952358365,276.21958166485865
17

/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,2956.43212890625,3510.448650907725,223.22570128541926
20,3002.90966796875,3513.879244878888,205.8811201680658
30,3102.814208984375,3526.1851904336363,170.58567837739676
40,3153.9794921875,3558.729785129428,163.08298471142018
50,3118.917236328125,3597.9517283989117,193.01375714589474
60,3118.93017578125,3626.925584703684,204.68276107870605
70,3129.5439453125,3660.695001769811,214.01269160319168
80,3047.384521484375,3664.137549549341,248.5036487943249
90,3097.17626953125,3673.4014877444133,232.17408385094092
100,3129.350341796875,3662.1954504549503,214.69526328745047
110,3137.68603515625,3678.8238947615027,218.0365989196107
120,3133.40966796875,3701.12289134413,228.74440993049964
130,3128.1259765625,3712.027952482924,235.26722197001837
140,3126.9140625,3722.9893785193563,240.17213413897866
150,3141.00146484375,3735.6601020768285,239.6013224339043
160,3179.23095703125,3765.42110372521,236.18917737260836
170

/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,3169.349609375,3735.6714481450617,228.18378981910323
20,3261.55224609375,3768.6760488152504,204.33157136193518
30,3344.563720703125,3779.7969502955675,175.36524460949596
40,3419.7138671875,3806.0556176155806,155.6657694315389
50,3406.3759765625,3867.514477953315,185.80306051305422
60,3365.05224609375,3889.8078096117824,211.435803839486
70,3383.423095703125,3929.8974762670696,220.1868030852279
80,3399.04443359375,3944.1038111895323,219.61666660491525
90,3382.6826171875,3965.2610060321167,234.73391900616096
100,3387.20068359375,3949.2214501947165,226.450791229536
110,3405.36962890625,3983.804991580546,233.06460069960104
120,3411.34765625,4015.8511381819844,243.5680315024715
130,3437.51318359375,3995.6927140131593,224.90307088839293
140,3445.8759765625,4004.571036219597,225.1107891983094
150,3422.742919921875,4002.8371777702123,233.73300682808002
160,3435.213134765625,4025.922362167388,238.010016499428
170,

In [5]:
plotter = pv.Plotter(shape=(len(initial_structure_gen_list), 2), notebook=True, window_size=(1000,2000))
for i,f in enumerate(initial_structure_gen_list):
    traj = md.load(f'initial_structures_tests/{f.__name__}/after_sim.cif')
    positions = traj.xyz
    mesh = pv.PolyData(positions[0])
    plotter.subplot(i,1)
    plotter.add_text(f"{f.__name__}, after simulation", font_size=10) 
    plotter.add_mesh(mesh, color="blue", point_size=5)
    lines = pv.lines_from_points(positions[0])
    plotter.add_mesh(lines, color="red", line_width=2)

    traj = md.load(f'initial_structures_tests/{f.__name__}/init_struct.cif')
    positions = traj.xyz
    mesh = pv.PolyData(positions[0])
    plotter.subplot(i,0)
    plotter.add_text(f"{f.__name__}, initial structure", font_size=10) 
    plotter.add_mesh(mesh, color="blue", point_size=5)
    lines = pv.lines_from_points(positions[0])
    plotter.add_mesh(lines, color="red", line_width=2)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:55384/index.html?ui=P_0x7fdd41b24df0_1&reconnect=auto" class="pyvi…